<a href="https://colab.research.google.com/github/Victor-LE-GALL/Projet-Data-Analyst/blob/main/Data_Analyst_Dash_Plotly_Tennis__v2_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jupyter-dash   # pour utiliser ce code dans Google Colab

In [2]:
# import des librairies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# on récupère le fichier csv comprenant tous les matchs
path_drive_tennis='/content/drive/MyDrive/Tennis/'
df=pd.read_csv(path_drive_tennis+'Data_tennis.csv')

In [3]:
def calcul_par_annee(type_tournoi, reverse, cote_min, cote_max, tetes_de_serie, tour):   # fonction qui calcule le gain obtenu en pariant sur certains matchs uniquement (d'après les paramètres entrés)
  x=[]
  annees=[2012, 2013, 2014, 2015, 2016, 2017 ,2018, 2019, 2020, 2021, 2022]
  nb_paris=[]
  cote_moyenne=[]

  for k in annees:
    solde=0
    nb=0
    cote_moy=0
    for tete_serie in tetes_de_serie:
      for i in range(0, len(df)):
        if df.at[i, 'Tournoi_type']==type_tournoi and df.at[i, 'Tour']==tour and  int(df.at[i, 'Année'])==k:
          if reverse==0:
            # on parie sur la tete de serie
            if df.at[i, 'Cote_1']>cote_min and df.at[i, 'Cote_1']<cote_max and df.at[i, 'Joueur_1']==tete_serie and df.at[i, 'Joueur_2']!=tete_serie:
              nb=nb+1
              cote_moy=cote_moy+df.at[i, 'Cote_1']
              if df.at[i, 'Res_1']>df.at[i, 'Res_2']:
                #gagné
                solde=solde-1+df.at[i, 'Cote_1']
              else:
                #perdu
                solde=solde-1

            if df.at[i, 'Cote_2']>cote_min and df.at[i, 'Cote_2']<cote_max and df.at[i, 'Joueur_2']==tete_serie and df.at[i, 'Joueur_1']!=tete_serie:
              nb=nb+1
              cote_moy=cote_moy+df.at[i, 'Cote_2']
              if df.at[i, 'Res_2']>df.at[i, 'Res_1']:
                #gagné
                solde=solde-1+df.at[i, 'Cote_2']
              else:
                #perdu
                solde=solde-1
          else:
            # on parie sur l'autre joueur
            if df.at[i, 'Cote_1']>cote_min and df.at[i, 'Cote_1']<cote_max and df.at[i, 'Joueur_1']==tete_serie and df.at[i, 'Joueur_2']!=tete_serie:
              nb=nb+1
              cote_moy=cote_moy+df.at[i, 'Cote_2']
              if df.at[i, 'Res_2']>df.at[i, 'Res_1']:
                #gagné
                solde=solde-1+df.at[i, 'Cote_2']
              else:
                #perdu
                solde=solde-1

            if df.at[i, 'Cote_2']>cote_min and df.at[i, 'Cote_2']<cote_max and df.at[i, 'Joueur_2']==tete_serie and df.at[i, 'Joueur_1']!=tete_serie:
              nb=nb+1
              cote_moy=cote_moy+df.at[i, 'Cote_1']
              if df.at[i, 'Res_1']>df.at[i, 'Res_2']:
                #gagné
                solde=solde-1+df.at[i, 'Cote_1']
              else:
                #perdu
                solde=solde-1

    if nb>0:
      cote_moy=cote_moy/nb
    else:
      cote_moy=0
    x.append(round(solde, 2))
    nb_paris.append(nb)
    cote_moyenne.append(cote_moy)

  # on retourne le gain, le nombre de paris et la cote moyenne. Ces trois variables sont des listes avec une valeur par année de 2012 à 2022.
  return x, nb_paris, cote_moyenne

In [4]:
def rentable():

  cotes_min=[1.2, 1.5, 2, 2.5, 3, 3.5]
  tours=[64, 32, 16, 8, 4, 2, 1]
  for i1 in range(0, 2):  # on détermine si l'on mise sur le joueur tête de série (reverse=0) ou sur l'adversaire (reverse=1)
    reverse=i1
    for i2 in range(0, 3): # itération sur le type de tournoi
      type_tournoi=i2
      print(type_tournoi)
      for i3 in cotes_min :  # itération sur les cotes (par tranches de 0.5)
        cote_min=i3
        cote_max=cote_min+0.5
        for i4 in tours:  # itération sur le tour
          tour=i4
          for i5 in range(0, 40): # itération sur les têtes de série
            tetes_de_serie=[i5]

            x=calcul_par_annee(type_tournoi, reverse, cote_min, cote_max, tetes_de_serie, tour) # calcul du gain grâce à la fonction calcul_par_annee

            # on garde les paramètres qui montrent un gain au moins sur 5 années et une perte au maximum 1 fois (certaines années n'ont aucun paris, donc un résultat nul)
            pos=0
            neg=0
            for i in range(0, len(x)):
              if x[i]>0:
                pos=pos+1
              if x[i]<0:
                neg=neg+1
            if pos>4 and neg <2:
              print(type_tournoi, reverse, cote_min, cote_max, tetes_de_serie, tour)
              print(x)
              

In [5]:
#à l'aide de la fonction "rentable", les paramètres suivants ont été identifiés comme rapportant un gain
# l'objectif est maintenant d'en vérifier la stabilité, un par un (visuellement grâce à des graphiques)

#x1, x2... gain stable
#x1_1, x1_2... gain plutôt stable
#x2_1, x2_2... gain moyennement stable
#-----x... gain trop peu stable pour être considéré

x1, nb1, c1=calcul_par_annee(0, 0, 1.2, 1.7, [38], 16)
x2_1, nb2_1, c2_1=calcul_par_annee(0, 0, 1.2, 1.7, [6], 1)
x2_2, nb2_2, c2_2=calcul_par_annee(0, 0, 2, 2.5, [37], 16)
x2_3, nb2_3, c2_3=calcul_par_annee(0, 0, 2, 2.5, [3], 1)
#----x=calcul_par_annee(0, 0, 2, 2.5, [3], 2)
x1_1, nb1_1, c1_1=calcul_par_annee(0, 0, 3, 3.5, [8], 4)

x2,nb2, c2=calcul_par_annee(1, 0, 1.2, 1.7, [8], 32)
x2_4,nb2_4, c2_4=calcul_par_annee(1, 0, 1.2, 1.7, [16], 32)
#----x=calcul_par_annee(1, 0, 1.2, 1.7, [17], 32)
#----x=calcul_par_annee(1, 0, 1.2, 1.7, [19], 32)
x2_5, nb2_5, c2_5=calcul_par_annee(1, 0, 1.2, 1.7, [3], 2)
x2_6, nb2_6, c2_6=calcul_par_annee(1, 0, 1.5, 2, [34], 64)
x2_7, nb2_7, c2_7=calcul_par_annee(1, 0, 2, 2.5, [0], 4)
x2_8, nb2_8, c2_8=calcul_par_annee(1, 0, 3.5, 4, [0], 4)

x2_9, nb2_9, c2_9=calcul_par_annee(2, 0, 1.2, 1.7, [17], 64)
x2_10, nb2_10, c2_10=calcul_par_annee(2, 0, 1.2, 1.7, [22], 64)
x3, nb3, c3=calcul_par_annee(2, 0, 1.2, 1.7, [6], 32)
#---x=calcul_par_annee(2, 0, 1.2, 1.7, [9], 32)
x4, nb4, c4=calcul_par_annee(2, 0, 1.2, 1.7, [10], 32)
x2_11, nb2_11, c2_11=calcul_par_annee(2, 0, 1.2, 1.7, [28], 32)
#---x=calcul_par_annee(2, 0, 1.2, 1.7, [16], 16)
#---x=calcul_par_annee(2, 0, 1.2, 1.7, [20], 16)
x5, nb5, c5=calcul_par_annee(2, 0, 1.2, 1.7, [7], 8)
x2_12, nb2_12, c2_12=calcul_par_annee(2, 0, 1.2, 1.7, [8], 8)
#----x=calcul_par_annee(2, 0, 1.2, 1.7, [1], 4)
x1_2, nb1_2, c1_2=calcul_par_annee(2, 0, 1.2, 1.7, [7], 4)
x1_3, nb1_3, c1_3=calcul_par_annee(2, 0, 1.2, 1.7, [1], 2)
x6, nb6, c6=calcul_par_annee(2, 0, 1.2, 1.7, [4], 2)


x1_4, nb1_4, c1_4=calcul_par_annee(0, 1, 1.2, 1.7, [15], 16)
x1_5, nb1_5, c1_5=calcul_par_annee(0, 1, 1.2, 1.7, [13], 4)
x7, nb7, c7=calcul_par_annee(0, 1, 2.5, 3, [7], 8)
x1_6, nb1_6, c1_6=calcul_par_annee(0, 1, 2.5, 3, [6], 4)
x8, nb8, c8=calcul_par_annee(0, 1, 3, 3.5, [0], 32)
x9, nb9, c9=calcul_par_annee(0, 1, 3, 3.5, [35], 32)
x2_13, nb2_13, c2_13=calcul_par_annee(0, 1, 3, 3.5, [7], 16)
x2_14, nb2_14, c2_14=calcul_par_annee(0, 1, 3, 3.5, [36], 16)
#----x=calcul_par_annee(0, 1, 3, 3.5, [5], 4)
x1_7, nb1_7, c1_7=calcul_par_annee(0, 1, 3, 3.5, [6], 4)
x10, nb10, c10=calcul_par_annee(0, 1, 3, 3.5, [0], 32)
#----x=calcul_par_annee(0, 1, 3, 3.5, [7], 4)

x11 ,nb11, c11=calcul_par_annee(1, 1, 1.2, 1.7, [21], 32)
x1_8, nb1_8, c1_8=calcul_par_annee(1, 1, 1.2, 1.7, [11], 8)
x12, nb12, c12=calcul_par_annee(1, 1, 1.5, 2, [8], 16)
x13, nb13, c13=calcul_par_annee(1, 1, 2, 2.5, [13], 16)
x2_15, nb2_15, c2_15=calcul_par_annee(1, 1, 2, 2.5, [7], 8)
#----x=calcul_par_annee(1, 1, 3.5, 4, [0], 8)

x1_9, nb1_9, c1_9=calcul_par_annee(2, 1, 1.2, 1.7, [13], 32)
x2_16, nb2_16, c2_16=calcul_par_annee(2, 1, 1.2, 1.7, [11], 16)
x1_10, nb1_10, c1_10=calcul_par_annee(2, 1, 1.5, 2, [30], 32)
x2_17, nb2_17, c2_17=calcul_par_annee(2, 1, 3, 3.5, [0], 8)
x1_11, nb1_11, c1_11=calcul_par_annee(2, 1, 3.5, 4, [35], 64)
x14, nb14, c14=calcul_par_annee(2, 1, 3.5, 4, [0], 8)


In [6]:
def auto_bar_data(starting_year, ending_year): #calcul des données nécessaires au graphique en barres à partir de l'année de début et l'année de fin
  series = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
  starting_year=str(starting_year)
  ending_year=str(ending_year)
  start=series.index(starting_year)
  end=series.index(ending_year)

  difference=abs(int(start)-int(end))+1
  nb=[0]*difference
  for j in range(0, difference):
    i=j+start
    nb[j]=nb1[i]+nb2[i]+nb3[i]+nb4[i]+nb5[i]+nb6[i]+nb7[i]+nb8[i]+nb9[i]+nb10[i]+nb11[i]+nb12[i]+nb13[i]+nb14[i] + nb1_1[i]+nb1_2[i]+nb1_3[i]+nb1_4[i]+nb1_5[i]+nb1_6[i]+nb1_7[i]+nb1_8[i]+nb1_9[i]+nb1_10[i]+nb1_11[i]+nb2_1[i]+nb2_2[i]+nb2_3[i]+nb2_4[i]+nb2_5[i]+nb2_6[i]+nb2_7[i]+nb2_8[i]+nb2_9[i]+nb2_10[i]+nb2_11[i]+nb2_12[i]+nb2_13[i]+nb2_14[i]+nb2_15[i]+nb2_16[i]+nb2_17[i]

  series= [i for i in range(int(starting_year), int(ending_year)+1)]
  for i in range(0, len(series)):
    series[i]=str(series[i])
  return series, nb
  

In [7]:
def auto_bar_2_data(starting_year, ending_year): #calcul des données nécessaires au graphique en barres à partir de l'année de début et l'année de fin
  series = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
  starting_year=str(starting_year)
  ending_year=str(ending_year)
  start=series.index(starting_year)
  end=series.index(ending_year)

  difference=abs(int(start)-int(end))+1
  c=[0]*difference


  liste_c=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14, c1_1,c1_2,c1_3,c1_4,c1_5,c1_6,c1_7,c1_8,c1_9,c1_10,c1_11,c2_1,c2_2,c2_3,c2_4,c2_5,c2_6,c2_7,c2_8,c2_9,c2_10,c2_11,c2_12,c2_13,c2_14,c2_15,c2_16,c2_17]
  for k in range(0, len(c)):
    i=k+start
    nb_cote=0
    for j in liste_c:
      if j[i]>0.5:
        nb_cote=nb_cote+1

    calcul=c1[i]+c2[i]+c3[i]+c4[i]+c5[i]+c6[i]+c7[i]+c8[i]+c9[i]+c10[i]+c11[i]+c12[i]+c13[i]+c14[i]+ c1_1[i]+c1_2[i]+c1_3[i]+c1_4[i]+c1_5[i]+c1_6[i]+c1_7[i]+c1_8[i]+c1_9[i]+c1_10[i]+c1_11[i]+c2_1[i]+c2_2[i]+c2_3[i]+c2_4[i]+c2_5[i]+c2_6[i]+c2_7[i]+c2_8[i]+c2_9[i]+c2_10[i]+c2_11[i]+c2_12[i]+c2_13[i]+c2_14[i]+c2_15[i]+c2_16[i]+c2_17[i]
    c[k]=calcul/nb_cote

  series= [i for i in range(int(starting_year), int(ending_year)+1)]
  for i in range(0, len(series)):
    series[i]=str(series[i])

  return series, c


In [8]:
def auto_pie_data(starting_year, ending_year): #calcul des données nécessaires au diagramme circulaire à partir de l'année de début et l'année de fin
  series = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
  starting_year=str(starting_year)
  ending_year=str(ending_year)
  start=series.index(starting_year)
  end=series.index(ending_year)

  difference=abs(int(start)-int(end))+1
  x_gc=[0]*difference
  x_m1=[0]*difference
  x_atp=[0]*difference


  for k in range(0, difference):
    i=k+start
    x_atp[k]=x1[i]+x1_1[i]+x2_1[i]+x2_2[i]+x2_3[i]+x7[i]+x8[i]+x9[i]+x10[i]+ x1_5[i]+x1_6[i]+x1_7[i]+x2_13[i] + x2_14[i]   
    x_m1[k]=x2[i]+x2_4[i]+x2_5[i]+x2_6[i]+x2_7[i]+x2_8[i] +x11[i]+x12[i]+x13[i]+ x1_8[i]+x2_15[i]   
    x_gc[k]=x3[i]+x4[i]+x5[i]+x6[i] +x1_2[i]+x1_3[i]+ x2_9[i]+x2_10[i]+x2_11[i]+ x2_12[i] +x1_9[i]+x2_16[i]+x1_10[i] + x2_17[i]+x1_11[i]+x14[i]     



  s1=(sum(x_atp))
  s2=(sum(x_m1))
  s3=(sum(x_gc))

  prct1=s1/(s1+s2+s3)
  prct2=s2/(s1+s2+s3)
  prct3=s3/(s1+s2+s3)

  sizes_pie = [prct1, prct2, prct3]
  labels_pie = 'ATP 500 et moins', 'Masters 1000', 'Grand Chelem'

  return sizes_pie, labels_pie


In [9]:
def auto_multi_bar_data(starting_year, ending_year): #calcul des données nécessaires au graphique en barres multiples à partir de l'année de début et l'année de fin
  series = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
  starting_year=str(starting_year)
  ending_year=str(ending_year)
  start=series.index(starting_year)
  end=series.index(ending_year)

  difference=abs(int(start)-int(end))+1

  nb_gc=[0]*difference
  nb_m1=[0]*difference
  nb_atp=[0]*difference

  for j in range(0, difference):
    i=j+start

    nb_atp[j]=nb1[i]+nb7[i]+nb8[i]+nb9[i]+nb10[i]+ nb1_1[i]+nb1_5[i]+nb1_6[i]+nb1_7[i]+nb2_1[i]+nb2_2[i]+nb2_3[i]+nb2_13[i]+nb2_14[i]
    nb_m1[j]=nb2[i]+nb11[i]+nb12[i]+nb13[i]+nb1_8[i]+nb2_4[i]+nb2_5[i]+nb2_6[i]+nb2_7[i]+nb2_8[i]+nb2_15[i]
    nb_gc[j]=nb3[i]+nb4[i]+nb5[i]+nb6[i]+nb14[i] + nb1_2[i]+nb1_3[i]+nb1_9[i]+nb1_10[i]+nb1_11[i]+nb2_9[i]+nb2_10[i]+nb2_11[i]+nb2_12[i]+nb2_16[i]+nb2_17[i]
    

  series= [i for i in range(int(starting_year), int(ending_year)+1)]
  for i in range(0, len(series)):
    series[i]=str(series[i])
  return series, nb_gc, nb_m1, nb_atp


In [10]:
def auto_bar_h_data(starting_year, ending_year): #calcul des données nécessaires au graphique en barres horizontales à partir de l'année de début et l'année de fin
  series = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
  starting_year=str(starting_year)
  ending_year=str(ending_year)
  start=series.index(starting_year)
  end=series.index(ending_year)

  difference=abs(int(start)-int(end))+1

  c_gc=[0]*difference
  c_m1=[0]*difference
  c_atp=[0]*difference


  liste_c_gc=[c1,c7,c8,c9,c10, c1_1, c1_5,c1_6,c1_7, c2_1,c2_2,c2_3,c2_13,c2_14]
  liste_c_m1=[c2,c11,c12,c13, c1_8, c2_4,c2_5,c2_6,c2_7,c2_8, c2_15]
  liste_c_atp=[c3,c4,c5,c6, c14, c1_2,c1_3, c1_9,c1_10,c1_11, c2_9,c2_10,c2_11,c2_12, c2_16,c2_17]

  for ik in range(0, difference):
    i=ik+start
    nb_cote_gc=0
    nb_cote_m1=0
    nb_cote_atp=0
    for j in liste_c_gc:
      if j[i]>0.5:
        nb_cote_gc=nb_cote_gc+1
    for k in liste_c_m1:
      if k[i]>0.5:
        nb_cote_m1=nb_cote_m1+1
    for l in liste_c_atp:
      if l[i]>0.5:
        nb_cote_atp=nb_cote_atp+1


    calcul_atp=c1[i]+c1_1[i]+c2_1[i]+c2_2[i]+c2_3[i]+c7[i]+c8[i]+c9[i]+c10[i]+ c1_5[i]+c1_6[i]+c1_7[i]+c2_13[i] + c2_14[i]   
    calcul_m1=c2[i]+c2_4[i]+c2_5[i]+c2_6[i]+c2_7[i]+c2_8[i] +c11[i]+c12[i]+c13[i]+ c1_8[i]+c2_15[i]   
    calcul_gc=c3[i]+c4[i]+c5[i]+c6[i] +c1_2[i]+c1_3[i]+ c2_9[i]+c2_10[i]+c2_11[i]+ c2_12[i] +c1_9[i]+c2_16[i]+c1_10[i] + c2_17[i]+c1_11[i]+c14[i] 

    if nb_cote_gc>0:
      c_gc[ik]=calcul_gc/nb_cote_gc
    else:
      c_gc[ik]=0

    if nb_cote_m1>0:
      c_m1[ik]=calcul_m1/nb_cote_m1
    else:
      c_m1[ik]=0

    if nb_cote_atp>0:
      c_atp[ik]=calcul_atp/nb_cote_atp
    else:
      c_atp[ik]=0
    
  total=0
  nombre=0
  for i in range(0, len(c_atp)):
    total=total+c_atp[i]
    if c_atp[i]>0:
      nombre=nombre+1
  s1=total/nombre
  total=0
  nombre=0
  for i in range(0, len(c_m1)):
    total=total+c_m1[i]
    if c_m1[i]>0:
      nombre=nombre+1
  s2=total/nombre
  total=0
  nombre=0
  for i in range(0, len(c_gc)):
    total=total+c_gc[i]
    if c_gc[i]>0:
      nombre=nombre+1
  s3=total/nombre

  c_tot=[]
  c_tot.append(s1)
  c_tot.append(s2)
  c_tot.append(s3)
  series_bar_h = ['ATP 500 ou moins', 'Masters 1000', 'Grand Chelem']

  return series_bar_h, c_tot


In [11]:
def auto_bar_line_data(starting_year, ending_year): #calcul des données nécessaires à la courbe à partir de l'année de début et l'année de fin
  series = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
  starting_year=str(starting_year)
  ending_year=str(ending_year)
  start=series.index(starting_year)
  end=series.index(ending_year)

  difference=abs(int(start)-int(end))+1
  x_bar=[0]*difference
  x_tot=[0]*difference

  total=0
  for k in range(0, difference):
    i=k+start
    x_bar[k]=x1[i]+x2[i]+x3[i]+x4[i]+x5[i]+x6[i]+x7[i]+x8[i]+x9[i]+x10[i]+x11[i]+x12[i]+x13[i]+x14[i] + x1_1[i]+x1_2[i]+x1_3[i]+x1_4[i]+x1_5[i]+x1_6[i]+x1_7[i]+x1_8[i]+x1_9[i]+x1_10[i]+x1_11[i]+x2_1[i]+x2_2[i]+x2_3[i]+x2_4[i]+x2_5[i]+x2_6[i]+x2_7[i]+x2_8[i]+x2_9[i]+x2_10[i]+x2_11[i]+x2_12[i]+x2_13[i]+x2_14[i]+x2_15[i]+x2_16[i]+x2_17[i]

  for i in range(0, len(x_bar)):
    total=total+x_bar[i]
    x_tot[i]=total

  series= [i for i in range(int(starting_year), int(ending_year)+1)]
  for i in range(0, len(series)):
    series[i]=str(series[i])

  return series, x_bar, x_tot

In [ ]:
# import des librairies nécessaires à la création du dashboard
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State


# Création d'une app
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

# Liste des années (de 2012 à 2022)
year_list = [i for i in range(2012, 2022, 1)]


# Layout de l'application
app.layout = html.Div(children=[ 
                                # Création du titre du dashboard
                                html.H1('Tennis Results from 2012 to 2022:', style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),
    
                                
                                # Création d'une division
                                html.Div([
                                    
                                    html.Div([
                                        # Création d'une division avec une liste déroulante 
                                        html.Div(
                                            [
                                             # titre de la liste déroulante
                                            html.H2('Tennis Options:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        # Création d'une liste déroulante pour choisir une option
                                        dcc.Dropdown(id='input-type', 
                                                options=[
                                                        {'label': 'Tennis Option 1', 'value': 'OPT1'},
                                                        {'label': 'Tennis Option 2', 'value': 'OPT2'}
                                                        ],
                                                placeholder='Select a report type',
                                                
                                                style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                        
                                    
                                    ], style={'display':'flex'}),
                                    
                                   # Nouvelle division 
                                   html.Div([
                                       # Création d'une liste déroulante pour choisir l'année de départ
                                        html.Div(
                                            [
                                            html.H2('Choose starting Year:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-starting-year', 
                                                     
                                                     options=[{'label': i, 'value': i} for i in year_list],
                                                     placeholder="Select a year",
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            
                                            ], style={'display': 'flex'}),  
                                          
                                    html.Div([
                                          # Création d'une liste déroulante pour choisir l'année de départ
                                            html.Div(
                                                [
                                                html.H2('Choose Ending Year:', style={'margin-right': '2em'})
                                                ]
                                            ),
                                            dcc.Dropdown(id='input-ending-year', 
                                                        
                                                        options=[{'label': i, 'value': i} for i in year_list],
                                                        placeholder="Select a year",
                                                        style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                                
                                                ], style={'display': 'flex'}),  
                                              ]),
                                
                                # Création de deux divisions avec des emplacements pour les graphiques (vides pour l'instant car créé après)
    
                                html.Div([
                                        html.Div([ ], id='plot2'),
                                        html.Div([ ], id='plot1')
                                ], style={'display': 'flex'}),
                                
                                html.Div([

                                    html.Div([ ], id='plot4'),

                                    html.Div([ ], id='plot5'),

                                    html.Div([ ], id='plot3'),
    

                                    ], style={'display': 'flex'}),
                               
                                ])

# Définition d'une "Callback function". Les outputs sont les graphiques, ils sont modifiés/mis à jour lorsque les inputs (listes déroulantes) sont choisis par l'utilisateur
@app.callback( [Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children'),
                Output(component_id='plot3', component_property='children'),
                Output(component_id='plot4', component_property='children'),
                Output(component_id='plot5', component_property='children')],

               [Input(component_id='input-type', component_property='value'),
                Input(component_id='input-starting-year', component_property='value'),
                Input(component_id='input-ending-year', component_property='value')],
               [State("plot1", 'children'), State("plot2", "children"),
                State("plot3", "children"), State("plot4", "children"),
                State("plot5", "children")
               ])
# Fonction qui permet de tracer les graphiques en fonction des données d'entrée (choisies par l'utilisateur sur l'interface)
def get_graph(chart, starting_year, ending_year, children1, children2, c3, c4, c5):

        if chart == 'OPT1':
            #on calcule les données nécessaires à l'affichage des graphiques
            series_bar_line, y_bar, y_line=auto_bar_line_data(starting_year, ending_year)
            bar_fig = px.bar(x=series_bar_line, y=y_bar, title='Gain par an')
            bar_fig.update_layout(xaxis_title="",yaxis_title="Gain pour une mise unitaire", title_x=0.5)

            line_fig = px.line(x=series_bar_line, y=y_line, title='Evolution du gain total')
            line_fig.update_layout(xaxis_title="",yaxis_title="Gain pour une mise unitaire" , title_x=0.5)

            sizes_pie, labels_pie=auto_pie_data(starting_year, ending_year)
            pie_fig = px.pie(values=sizes_pie, names=labels_pie, title='Proportion des gains selon le type de tournoi')
            line_fig.update_layout(title_x=0.5)

            multi_bar_series, x1, x2, x3=auto_multi_bar_data(starting_year, ending_year)
            df_multi_bar = pd.DataFrame({'Masters 1000': x1, 'ATP 500 et moins':x2, 'Grand Chelem': x3, 'Indd': multi_bar_series})
            multi_bar_fig = px.bar(df_multi_bar, x='Indd', y=['Masters 1000', 'ATP 500 et moins', 'Grand Chelem'], labels={'Masters 1000': "1", 'ATP 500 et moins': '2', 'Grand Chelem':'3'}, title='Nombre de paris par an et par type de tournoi')
            multi_bar_fig.update_layout(xaxis_title="",yaxis_title="", legend_title="", title_x=0.5)

            bar_h_series, bar_h_values=auto_bar_h_data(starting_year, ending_year)
            bar_h_fig = px.bar(x=bar_h_values, y=bar_h_series, title='Cote moyenne par type de tournoi', orientation='h')
            bar_h_fig.update_layout(xaxis_title="Cote moyenne",yaxis_title="", title_x=0.5)
            
            
            # On retourne les différents graphiques
            return [dcc.Graph(figure=bar_h_fig ,style={'width': '150vh', 'height': '70vh'}), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=multi_bar_fig ,style={'width': '90vh', 'height': '70vh'}),
                    dcc.Graph(figure=bar_fig ,style={'width': '80vh', 'height': '70vh'}),
                    dcc.Graph(figure=line_fig ,style={'width': '80vh', 'height': '70vh'})

                   
                   ]
        else: # ici, l'option 2 n'ajoute par de fonctionnalité (copie de l'option 1) et sert simplement d'exemple pour montrer les possibilités de ce type de code.
            series_bar_line, y_bar, y_line=auto_bar_line_data(starting_year, ending_year)
            bar_fig = px.bar(x=series_bar_line, y=y_bar, title='Gain par an')
            bar_fig.update_layout(xaxis_title="",yaxis_title="Gain pour une mise unitaire")

            line_fig = px.line(x=series_bar_line, y=y_line, title='Evolution du gain total')
            line_fig.update_layout(xaxis_title="",yaxis_title="Gain pour une mise unitaire")

            sizes_pie, labels_pie=auto_pie_data(starting_year, ending_year)
            pie_fig = px.pie(values=sizes_pie, names=labels_pie, title='Proportion des gains selon le type de tournoi')

            multi_bar_series, x1, x2, x3=auto_multi_bar_data(starting_year, ending_year)
            df_multi_bar = pd.DataFrame({'Masters 1000': x1, 'ATP 500 et moins':x2, 'Grand Chelem': x3, 'Indd': multi_bar_series})
            multi_bar_fig = px.bar(df_multi_bar, x='Indd', y=['Masters 1000', 'ATP 500 et moins', 'Grand Chelem'], labels={'Masters 1000': "1", 'ATP 500 et moins': '2', 'Grand Chelem':'3'}, title='Nombre de paris par an et par type de tournoi')
            multi_bar_fig.update_layout(xaxis_title="",yaxis_title="", legend_title="",)

            bar_h_series, bar_h_values=auto_bar_h_data(starting_year, ending_year)
            bar_h_fig = px.bar(x=bar_h_values, y=bar_h_series, title='Cote moyenne par type de tournoi', orientation='h')
            bar_h_fig.update_layout(xaxis_title="Cote moyenne",yaxis_title="")
            
            return [dcc.Graph(figure=bar_h_fig), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=multi_bar_fig),
                    dcc.Graph(figure=bar_fig),
                    dcc.Graph(figure=line_fig)
                   ]

# On lance l'application
app.run_server(mode='inline')